#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

# Dataset Loading and Pre-Processing

In [2]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [4]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
11218,17.0,85,14.5,95.0,overcast,992.41,2,Autumn,4.0,inland,Rainy
2932,31.0,75,8.0,80.0,partly cloudy,1026.17,9,Summer,8.0,mountain,Sunny
9104,20.0,62,3.0,18.0,clear,1012.83,10,Spring,5.0,inland,Sunny
6870,-6.0,40,8.0,75.0,cloudy,1105.31,5,Autumn,5.0,coastal,Rainy
10016,28.0,66,3.0,26.0,partly cloudy,1006.05,4,Winter,5.5,coastal,Cloudy


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [5]:
# START CODE HERE
object_columns = data.select_dtypes(include='object').columns.tolist()
non_object_columns = data.select_dtypes(exclude='object').columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [6]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [7]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE
column_transformer = ColumnTransformer(
    transformers=[
        ('obj', OrdinalEncoder(), object_columns),
        ('num', StandardScaler(), non_object_columns)
    ],
    remainder='passthrough'  # If we want to keep columns not specified in transformers
)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)
# END CODE HERE


all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [8]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [9]:
X = data_scaled_labeled.drop(columns=['Weather Type'])
y = data_scaled_labeled['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [10]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [11]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,3.0,2.0,1.0,-0.179894,0.410487,-0.048086,-1.147097,0.060275,-0.260799,1.049153
12278,3.0,3.0,1.0,0.222737,0.311445,0.458541,-0.176689,0.332600,-0.520104,0.900845
2859,2.0,3.0,2.0,-1.330267,0.113362,-1.350840,1.419788,-0.233826,-1.038715,-0.878846
5495,2.0,3.0,2.0,-1.617860,1.499948,-0.554712,0.073739,-0.353187,-0.779410,-0.285616
8469,2.0,3.0,1.0,-1.502823,1.202822,-0.120461,0.198953,-0.274150,-0.779410,-0.433923


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [19]:
from sklearn.ensemble import RandomForestClassifier

# Define the model
model = RandomForestClassifier(random_state=10)

model.fit(X_train, y_train)

# Evaluate the model on test data
score = model.score(X_test, y_test)

print(f"Model Score on Test Data: {score}")

Model Score on Test Data: 0.9174242424242425


**TEST** (DON'T EDIT THIS CELL)

In [20]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [22]:
input_data = X_test.head()
predictions = model.predict(input_data)


print("Input Data:")
print(input_data)



Input Data:
       Cloud Cover  Season  Location  Temperature  Humidity  Wind Speed  \
11923          3.0     3.0       2.0    -0.467487  0.113362   -0.771838   
2331           3.0     3.0       1.0    -1.387785  0.707613    0.169040   
217            3.0     3.0       2.0     0.452811 -0.877057    0.024290   
1760           3.0     2.0       0.0    -0.122375 -0.877057   -0.265211   
3963           3.0     0.0       0.0    -0.525006 -0.877057   -0.771838   

       Precipitation (%)  Atmospheric Pressure  UV Index  Visibility (km)  
11923          -0.990580              0.248187 -0.779410         0.455922  
2331            0.480684             -0.669331 -1.038715        -0.582231  
217            -0.990580             -0.041343 -0.779410         0.307615  
1760           -1.021883              0.008659 -0.520104         0.455922  
3963           -0.708848              0.073447 -0.001493         1.049153  


In [23]:
y = model.predict(input_data)
print("\nPredictions:")
print(predictions)


Predictions:
[0. 2. 0. 0. 0.]


Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.
